In [2]:
print(f'Start Spark name:{spark._sc.appName}, version:{spark.version}')

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Start Spark name:livy-session-5644, version:3.3.2.3.3.7190.2-1

In [3]:
%%info

ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
5639,application_1713270977862_6333,pyspark,idle,Link,Link,None,
5640,application_1713270977862_6334,pyspark,idle,Link,Link,None,
5643,application_1713270977862_6338,pyspark,idle,Link,Link,None,
5644,application_1713270977862_6339,pyspark,idle,Link,Link,None,✔


In [4]:
import os
print(f"remote USER={os.getenv('USER',None)}")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

remote USER=livy

In [5]:
%%local
import os
print(f"local USER={os.getenv('USER',None)}")

local USER=leclercq


In [6]:
%%local
import os
username=os.getenv('USER', 'anonymous')
hadoop_fs=os.getenv('HADOOP_DEFAULT_FS', 'hdfs://iccluster067.iccluster.epfl.ch:8020')
print(f"local username={username}\nhadoop_fs={hadoop_fs}")

local username=leclercq
hadoop_fs=hdfs://iccluster067.iccluster.epfl.ch:8020


In [7]:
 # (prevent deprecated np.bool error since numpy 1.24, until a new version of pandas/Spark fixes this)
import numpy as np
np.bool = np.bool_

username=spark.conf.get('spark.executorEnv.USERNAME', 'anonymous')
hadoop_fs=spark.conf.get('spark.executorEnv.HADOOP_DEFAULT_FS','hdfs://iccluster067.iccluster.epfl.ch:8020')
print(f"remote username={username}\nhadoop_fs={hadoop_fs}")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

remote username=leclercq
hadoop_fs=hdfs://iccluster067.iccluster.epfl.ch:8020

In [7]:
spark.sql(f"ADD JARS {hadoop_fs}/data/jars/esri-geometry-api-2.2.4.jar")
spark.sql(f"ADD JARS {hadoop_fs}/data/jars/spatial-sdk-hive-2.2.0.jar")
spark.sql(f"ADD JARS {hadoop_fs}/data/jars/spatial-sdk-json-2.2.0.jar")
spark.sql("CREATE OR REPLACE TEMPORARY FUNCTION ST_Point AS 'com.esri.hadoop.hive.ST_Point'")
spark.sql("CREATE OR REPLACE TEMPORARY FUNCTION ST_SetSRID AS 'com.esri.hadoop.hive.ST_SetSRID'")
spark.sql("CREATE OR REPLACE TEMPORARY FUNCTION ST_Contains AS 'com.esri.hadoop.hive.ST_Contains'")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

DataFrame[]

# Define the region

In [8]:
region_name, region_id = "lausanne", 2056

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

# Import data

In [9]:
# import data
istdaten = spark.sql(f"""
SELECT * FROM com490.sbb_orc_istdaten WHERE SUBSTR(bpuic, 1, 7) IN (SELECT DISTINCT stop_id FROM com490final.sbb_orc_stops)
""", header=True) # select only bpuic in common with stop_id
stops = spark.sql(f"SELECT * FROM com490final.sbb_orc_stops WHERE year = 2024 and month = 1", header=True)
stop_times = spark.sql(f"SELECT * FROM com490final.sbb_orc_stop_times WHERE year = 2024 and month = 1", header=True)
calendar = spark.sql(f"SELECT * FROM com490final.sbb_orc_calendar WHERE year = 2024 and month = 1", header=True)
regions = spark.sql(f"SELECT * FROM com490final.geo_shapes", header=True)
trips = spark.sql(f"SELECT * FROM com490final.sbb_orc_trips", header=True)

# show format
print(f"observations istdaten: {istdaten.count()}")
print(istdaten.show(n=3))

print(f"observations stops: {stops.count()}")
print(stops.show(n=3))

print(f"observations stop_times: {stop_times.count()}")
print(stop_times.show(n=3))

print(f"observations calendar: {calendar.count()}")
print(calendar.show(n=3))

print(f"observations regions: {regions.count()}")
print(regions.show(n=3))

print(f"observations trips: {trips.count()}")
print(trips.show(n=3))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

observations istdaten: 2886464689
+-----------+----------------+------------+-------------+--------------------+----------+----------+-----------+---------+-------------------+--------------+-------------+-------+-----------------+----------------+-------------------+------------------+----------------+-------------------+------------------+-------------+----+-----+
|betriebstag|fahrt_bezeichner|betreiber_id|betreiber_abk|      betreiber_name|produkt_id| linien_id|linien_text|umlauf_id|verkehrsmittel_text|zusatzfahrt_tf|faellt_aus_tf|  bpuic|haltestellen_name|    ankunftszeit|        an_prognose|an_prognose_status|    abfahrtszeit|        ab_prognose|ab_prognose_status|durchfahrt_tf|year|month|
+-----------+----------------+------------+-------------+--------------------+----------+----------+-----------+---------+-------------------+--------------+-------------+-------+-----------------+----------------+-------------------+------------------+----------------+-------------------+------

# Create datasets

In [10]:
# import spark functions
from pyspark.sql import functions as F
from pyspark.sql import Window

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [11]:
def get_region(regions, id=2056): # Lausanne
    region_df = regions.filter(F.col("objectid") == id)
    return region_df

def stops_in_region(region, stops):
    return stops.join(region, F.expr("ST_Contains(geometry, ST_Point(stop_lon, stop_lat))"))

def get_time(col:str, data):
    # get time in format HH:MM:SS
    temp = data.withColumn(col, F.split(col, " ").getItem(1))
    return temp

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [12]:
def create_data_graph(stop_data):
    data_prediction = stop_data.alias("data_prediction")
    
    # create stop sequence column
    windowSpec = Window.partitionBy('fahrt_bezeichner').orderBy('abfahrtszeit')
    data_prediction = data_prediction.withColumn('stop_sequence', F.row_number().over(windowSpec))
    
    # make each row a (stop_from, stop_to), i.e., an edge
    trip_window = Window.partitionBy(['fahrt_bezeichner']).orderBy('stop_sequence')
    
    df = data_prediction.withColumn("stop_id_from", F.lag('stop_id', 1).over(trip_window))
    df = df.withColumn("departure_time", F.lag('abfahrtszeit', 1).over(trip_window))
    df = df.withColumn("stop_name_from", F.lag('stop_name', 1).over(trip_window))
    
    df = df.withColumnRenamed("stop_id", "stop_id_to")
    df = df.withColumnRenamed("stop_name", "stop_name_to")
    
    # Only consider edges with a source different than the destination
    df = df.filter(df.stop_id_from != df.stop_id_to)
    
    # Remove Null value we got from using lag
    df = df.filter(df.stop_id_from.isNotNull())
    df = df.filter(df.departure_time.isNotNull())
    df = df.filter(df.stop_name_from.isNotNull())
    
    # Only keep the columns of interest
    journey = df.select("fahrt_bezeichner",
                        "stop_id_from", "stop_name_from",
                        "stop_id_to", "stop_name_to", 
                        "departure_time", "ankunftszeit",
                        "produkt_id")
    # journey = journey.withColumnRenamed("departure_time_from", "departure_time")
    journey = journey.withColumnRenamed("fahrt_bezeichner", "trip_id")
    journey = journey.withColumnRenamed("produkt_id", "transport_mode")
    journey = journey.withColumnRenamed("ankunftszeit", "arrival_time")
    
    
    return journey

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [13]:
def create_data_prediction(istdaten, stops_region, day, month, hour_min, hour_max, year):
    istdaten_specific = istdaten.withColumn("betriebstag", F.to_date("betriebstag", "dd.MM.yyyy"))
    istdaten_specific = istdaten_specific.filter(F.col("year") == year) # year
    istdaten_specific = istdaten_specific.filter(F.col("month") == month) # month
    istdaten_specific = istdaten_specific.filter(F.dayofweek(F.col("betriebstag")) == day) # mondays
    istdaten_specific = istdaten_specific.withColumn("day_cat", F.date_format("betriebstag", "EEEE"))
    istdaten_specific = istdaten_specific.withColumn("hour", F.hour(F.split("ankunftszeit", " ").getItem(1))) # get hours
    istdaten_specific = istdaten_specific.filter((F.col("hour") >= hour_min) & (F.col("hour") <= hour_max)) # truncate to reasonable hours
    istdaten_specific = istdaten_specific.withColumn("day", F.split(F.col("ankunftszeit"), " ").getItem(0).substr(1, 2))
    
    # select relevant columns
    istdaten_specific = istdaten_specific.select([
                                "fahrt_bezeichner", # trip id
                                "umlauf_id",
                                "linien_text", # line
                                "produkt_id", # mean of transport
                                "faellt_aus_tf", # cancelled (bool)
                                "bpuic", # id
                                "ankunftszeit", "an_prognose", # arrival time, true arrival time
                                "abfahrtszeit", "ab_prognose", # departure time, true departure time
                                "day_cat", "month", "hour"
    ])
    
    # get departure/arrival times only
    istdaten_formatted = istdaten_specific.alias("istdaten_formatted")
    
    istdaten_formatted = get_time("ankunftszeit", istdaten_formatted) # arrival time
    istdaten_formatted = get_time("an_prognose", istdaten_formatted) # actual arrival time
    istdaten_formatted = get_time("abfahrtszeit", istdaten_formatted) # arrival time
    istdaten_formatted = get_time("ab_prognose", istdaten_formatted) # actual arrival time
    
    # time format
    istdaten_formatted = istdaten_formatted.withColumn("ankunftszeit_sec", F.unix_timestamp(F.col("ankunftszeit"), "HH:mm"))
    istdaten_formatted = istdaten_formatted.withColumn("an_prognose_sec", F.unix_timestamp(F.col("an_prognose"), "HH:mm:ss"))
    istdaten_formatted = istdaten_formatted.withColumn("abfahrtszeit_sec", F.unix_timestamp(F.col("abfahrtszeit"), "HH:mm"))
    istdaten_formatted = istdaten_formatted.withColumn("ab_prognose_sec", F.unix_timestamp(F.col("ab_prognose"), "HH:mm:ss"))
    
    # compute delay (in minutes): expected arrival time at stop - actual arrival time at stop
    istdaten_formatted = istdaten_formatted.withColumn("delay", (F.col("an_prognose_sec") - F.col("ankunftszeit_sec"))/60)

    # remove null delay (otherwise not usable)
    istdaten_formatted = istdaten_formatted.dropna(subset=["delay"])
    
    # select relevant columns
    stops_region_data = stops_region.select(["stop_id", "stop_lat", "stop_lon", "stop_name", "parent_station"])
    
    # add geography (ie selects stops of istdaten in region) to istdaten
    df = istdaten_formatted.join(stops_region_data, F.substring(istdaten_formatted["bpuic"], 1, 7) == stops_region_data["stop_id"], how="inner")
    
    # add boolean delay 5 minutes
    df = df.withColumn("delay_bool", F.col("delay") >= 5)
    
    # reorder columns
    ordered_columns = ["fahrt_bezeichner", 'linien_text', 'parent_station', 'produkt_id', 'bpuic', 'stop_id',
                       'month', 'hour', 'day_cat',
                       "ankunftszeit", "an_prognose", # arrival time, true arrival time
                       "abfahrtszeit", "ab_prognose", # departure time, true departure time
                       "ankunftszeit_sec", "an_prognose_sec", # arrival time, true arrival time
                        "abfahrtszeit_sec", "ab_prognose_sec", # departure time, true departure time
                       'stop_lat', 'stop_lon', 'stop_name', 
                       'delay_bool', 'delay']
    df = df.select(ordered_columns)
    return df

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

# Execute code

In [14]:
# only keep stops in region
region_geometry = get_region(regions, region_id)
stops_region = stops_in_region(region_geometry, stops)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [15]:
# dataframe prediction
data_prediction = create_data_prediction(istdaten, stops_region, day=2, month=1, hour_min=6, hour_max=22, year=2024)
print(data_prediction.count())
data_prediction.show(n=5)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

1365180
+----------------+-----------+--------------+----------+-------+-------+-----+----+-------+------------+-----------+------------+-----------+----------------+---------------+----------------+---------------+---------------+--------------+------------------+----------+--------------------+
|fahrt_bezeichner|linien_text|parent_station|produkt_id|  bpuic|stop_id|month|hour|day_cat|ankunftszeit|an_prognose|abfahrtszeit|ab_prognose|ankunftszeit_sec|an_prognose_sec|abfahrtszeit_sec|ab_prognose_sec|       stop_lat|      stop_lon|         stop_name|delay_bool|               delay|
+----------------+-----------+--------------+----------+-------+-------+-----+----+-------+------------+-----------+------------+-----------+----------------+---------------+----------------+---------------+---------------+--------------+------------------+----------+--------------------+
|   85:55:104:000|        R20| Parent8501160|       Zug|8501160|8501160|    1|   6| Monday|       06:03|   06:02:31|      

In [16]:
# dataframe graph
data_graph = create_data_graph(data_prediction)
print(data_graph.count())
data_graph.show(n=5)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

88977
+--------------------+------------+--------------------+----------+--------------------+--------------+------------+--------------+
|             trip_id|stop_id_from|      stop_name_from|stop_id_to|        stop_name_to|departure_time|arrival_time|transport_mode|
+--------------------+------------+--------------------+----------+--------------------+--------------+------------+--------------+
|85:151:TL001-4506...|     8592114|   Lausanne, Sablons|   8591995|Lausanne, Bois de...|         16:15|       16:16|           Bus|
|85:151:TL001-4506...|     8591995|Lausanne, Bois de...|   8592068| Lausanne, Maladière|         16:16|       16:19|           Bus|
|85:151:TL001-4506...|     8592068| Lausanne, Maladière|   8592080|   Lausanne, Montoie|         16:19|       16:20|           Bus|
|85:151:TL001-4506...|     8592080|   Lausanne, Montoie|   8592110|Lausanne, Riant-Cour|         16:20|       16:22|           Bus|
|85:151:TL001-4506...|     8592110|Lausanne, Riant-Cour|   8591983| La

In [17]:
# stops_region_data = stops_region.select(["stop_id", "stop_lat", "stop_lon", "stop_name"])
# istdaten_region_data = istdaten.join(stops_region_data, F.substring(istdaten["bpuic"], 1, 7) == stops_region_data["stop_id"], how="inner")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [18]:
# istdaten_region_data.printSchema()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [27]:
columns = ["fahrt_bezeichner", "produkt_id", "linien_text", "abfahrtszeit", "ankunftszeit", "stop_id", "stop_lat", "stop_lon", "stop_name"]
istdaten_region_data = data_prediction.select(columns) # istdaten_region_data.select

# We want to rename the columns so that it corresponds to the other dataframes
istdaten_region_data = istdaten_region_data.withColumnRenamed("fahrt_bezeichner", "trip_id")
istdaten_region_data = istdaten_region_data.withColumnRenamed("produkt_id", "transport_mode")
# istdaten_region_data = istdaten_region_data.withColumnRenamed("linien_id", "line_id")
istdaten_region_data = istdaten_region_data.withColumnRenamed("linien_text", "line_text")
istdaten_region_data = istdaten_region_data.withColumnRenamed("abfahrtszeit", "departure_time")
istdaten_region_data = istdaten_region_data.withColumnRenamed("ankunftszeit", "arrival_time")

istdaten_region_data.show(n=5, truncate=False)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----------------------------+--------------+---------+--------------+------------+-------+---------------+-------------+------------------+
|trip_id                      |transport_mode|line_text|departure_time|arrival_time|stop_id|stop_lat       |stop_lon     |stop_name         |
+-----------------------------+--------------+---------+--------------+------------+-------+---------------+-------------+------------------+
|85:151:TL016-4506262507511760|Bus           |16       |06:48         |06:48       |8501207|46.521824762755|6.61355881936|Lausanne, Montelly|
|85:151:TL016-4506262507511760|Bus           |16       |06:48         |06:48       |8501207|46.521824762755|6.61355881936|Lausanne, Montelly|
|85:151:TL016-4506262507511760|Bus           |16       |06:48         |06:48       |8501207|46.521824762755|6.61355881936|Lausanne, Montelly|
|85:151:TL016-4506262507511760|Bus           |16       |06:48         |06:48       |8501207|46.521824762755|6.61355881936|Lausanne, Montelly|
|85:15

# Save datasets


Check existing files.

In [28]:
# livy
hdfs_path = f"/user/{username}/livy"

hadoop_conf = spark.sparkContext._jsc.hadoopConfiguration()
fs = spark._jvm.org.apache.hadoop.fs.FileSystem.get(hadoop_conf)

# List files in the directory
path = spark._jvm.org.apache.hadoop.fs.Path(hdfs_path)
file_status = fs.listStatus(path)
files = [file.getPath().getName() for file in file_status]

# Print the list of files
for file in files:
    print(file)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

coefficients_lr
coefficients_lr.csv
orc_data_bern_feb
orc_data_bern_feb.orc
orc_data_graph_one_day_lausanne
orc_data_pred_delay_jan_lausanne
orc_data_pred_delay_jan_lausanne.orc
orc_data_pred_delay_lausanne
orc_data_pred_delay_lausanne.orc
orc_data_train
orc_data_train.orc

In [9]:
# show tables in hive/hdfs
spark.sql(f"SHOW TABLES IN {username}").show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+---------+--------------------+-----------+
|namespace|           tableName|isTemporary|
+---------+--------------------+-----------+
| leclercq|    sbb_orc_istdaten|      false|
| leclercq|      sbb_orc_delays|      false|
| leclercq|   sbb_orc_delays_tb|      false|
| leclercq|       sbb_orc_stops|      false|
| leclercq| sbb_orc_delays_join|      false|
| leclercq|        epd_expr_csv|      false|
| leclercq|         epd_sam_csv|      false|
| leclercq|  sbb_stops_lausanne|      false|
| leclercq|          geo_shapes|      false|
| leclercq|   sbb_orc_one_month|      false|
| leclercq|sbb_stops_lausann...|      false|
| leclercq|sbb_stop_to_stop_...|      false|
| leclercq|sbb_stop_times_la...|      false|
| leclercq|           sbb_times|      false|
| leclercq|sbb_istdaten_stop...|      false|
| leclercq|   orc_data_bern_feb|      false|
| leclercq|orc_data_pred_del...|      false|
| leclercq|orc_data_graph_la...|      false|
| leclercq|orc_data_pred_del...|      false|
| leclercq

### Write files to livy

In [30]:
# write data
data_prediction.write.orc(path=f"/user/{username}/livy/orc_data_prediction_one_day_{region_name}", mode="overwrite")
data_graph.write.orc(path=f"/user/{username}/livy/orc_data_graph_one_day_{region_name}", mode="overwrite")
istdaten_region_data.write.orc(path=f"/user/{username}/livy/orc_data_istdaten_{region_name}", mode="overwrite")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [10]:
# read data
data_read = spark.read.orc(f"/user/{username}/livy/orc_data_prediction_one_day_{region_name}")
print(data_read.show(n=5))

data_read = spark.read.orc(f"/user/{username}/livy/orc_data_graph_one_day_{region_name}")
print(data_read.show(n=5))

data_read = spark.read.orc(f"/user/{username}/livy/orc_data_istdaten_{region_name}")
print(data_read.show(n=5))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------------+-----------+--------------+----------+-------+-------+-----+----+-------+------------+-----------+------------+-----------+----------------+---------------+----------------+---------------+---------------+--------------+-------------+----------+-------------------+
|fahrt_bezeichner|linien_text|parent_station|produkt_id|  bpuic|stop_id|month|hour|day_cat|ankunftszeit|an_prognose|abfahrtszeit|ab_prognose|ankunftszeit_sec|an_prognose_sec|abfahrtszeit_sec|ab_prognose_sec|       stop_lat|      stop_lon|    stop_name|delay_bool|              delay|
+----------------+-----------+--------------+----------+-------+-------+-----+----+-------+------------+-----------+------------+-----------+----------------+---------------+----------------+---------------+---------------+--------------+-------------+----------+-------------------+
|   85:55:104:000|        R20| Parent8501181|       Zug|8501181|8501181|    1|   6| Monday|       06:05|   06:04:08|        null|       null|       

### Write files to hdfs/hive

In [32]:
data_prediction.printSchema()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- fahrt_bezeichner: string (nullable = true)
 |-- linien_text: string (nullable = true)
 |-- parent_station: string (nullable = true)
 |-- produkt_id: string (nullable = true)
 |-- bpuic: string (nullable = true)
 |-- stop_id: string (nullable = true)
 |-- month: integer (nullable = true)
 |-- hour: integer (nullable = true)
 |-- day_cat: string (nullable = true)
 |-- ankunftszeit: string (nullable = true)
 |-- an_prognose: string (nullable = true)
 |-- abfahrtszeit: string (nullable = true)
 |-- ab_prognose: string (nullable = true)
 |-- ankunftszeit_sec: long (nullable = true)
 |-- an_prognose_sec: long (nullable = true)
 |-- abfahrtszeit_sec: long (nullable = true)
 |-- ab_prognose_sec: long (nullable = true)
 |-- stop_lat: double (nullable = true)
 |-- stop_lon: double (nullable = true)
 |-- stop_name: string (nullable = true)
 |-- delay_bool: boolean (nullable = true)
 |-- delay: double (nullable = true)

In [12]:
# create table to store data in HDFS
hdfs_path = f"/user/{username}/livy/orc_data_prediction_one_day_{region_name}"

spark.sql(f"DROP TABLE IF EXISTS {username}.orc_data_predictions_one_day_{region_name}")

spark.sql(f"""
CREATE EXTERNAL TABLE IF NOT EXISTS {username}.orc_data_prediction_one_day_{region_name}
(
    fahrt_bezeichner STRING,
    linien_text STRING,
    parent_station STRING,
    produkt_id STRING,
    bpuic STRING,
    stop_id STRING,
    month INT,
    hour INT,
    day_cat STRING,
    ankunftszeit STRING,
    an_prognose STRING,
    abfahrtszeit STRING,
    ab_prognose STRING,
    ankunftszeit_sec FLOAT,
    an_prognose_sec FLOAT,
    abfahrtszeit_sec FLOAT,
    ab_prognose_sec FLOAT,
    stop_lat FLOAT,
    stop_lon FLOAT,
    stop_name STRING,
    delay_bool BOOLEAN,
    delay FLOAT
)
STORED AS ORC
LOCATION '{hdfs_path}'
""")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

DataFrame[]

In [34]:
data_graph.printSchema()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- trip_id: string (nullable = true)
 |-- stop_id_from: string (nullable = true)
 |-- stop_name_from: string (nullable = true)
 |-- stop_id_to: string (nullable = true)
 |-- stop_name_to: string (nullable = true)
 |-- departure_time: string (nullable = true)
 |-- arrival_time: string (nullable = true)
 |-- transport_mode: string (nullable = true)

In [14]:
# create table to store data in HDFS
hdfs_path = f"/user/{username}/livy/orc_data_graph_one_day_{region_name}"

spark.sql(f"DROP TABLE IF EXISTS {username}.orc_data_graph_one_day_{region_name}")

spark.sql(f"""
CREATE EXTERNAL TABLE IF NOT EXISTS {username}.orc_data_graph_one_day_{region_name}
(
    trip_id STRING,
    stop_id_from STRING,
    stop_name_from STRING,
    stop_id_to STRING,
    stop_name_to STRING,
    departure_time STRING,
    arrival_time STRING,
    transport_mode STRING
)
STORED AS ORC
LOCATION '{hdfs_path}'
""")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

DataFrame[]

In [37]:
istdaten_region_data.printSchema()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- trip_id: string (nullable = true)
 |-- transport_mode: string (nullable = true)
 |-- line_text: string (nullable = true)
 |-- departure_time: string (nullable = true)
 |-- arrival_time: string (nullable = true)
 |-- stop_id: string (nullable = true)
 |-- stop_lat: double (nullable = true)
 |-- stop_lon: double (nullable = true)
 |-- stop_name: string (nullable = true)

In [16]:
hdfs_path = f"/user/{username}/livy/orc_data_istdaten_{region_name}"

spark.sql(f"DROP TABLE IF EXISTS {username}.orc_data_istdaten_{region_name}")

spark.sql(f"""
CREATE EXTERNAL TABLE IF NOT EXISTS {username}.orc_data_istdaten_{region_name}
(
    trip_id STRING,
    transport_mode STRING,
    line_text STRING,
    departure_time STRING,
    arrival_time STRING,
    stop_id STRING,
    stop_lat DOUBLE,
    stop_lon DOUBLE,
    stop_name STRING
)
STORED AS ORC
LOCATION '{hdfs_path}'
""")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

DataFrame[]

In [19]:
spark.stop()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…